In [1]:
import yfinance as yf
import pandas as pd

ticker = 'SPY'
data = yf.Ticker(ticker)

hist = data.history(period='max', auto_adjust=True)
df = pd.DataFrame(hist).reset_index()
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1993-01-29,25.818120,25.818120,25.689672,25.799770,1003200,0.0,0
1,1993-02-01,25.818125,25.983273,25.818125,25.983273,480500,0.0,0
2,1993-02-02,25.964916,26.056665,25.909866,26.038315,201300,0.0,0
3,1993-02-03,26.075019,26.331916,26.056669,26.313566,529400,0.0,0
4,1993-02-04,26.405305,26.478704,26.111709,26.423655,531500,0.0,0
...,...,...,...,...,...,...,...,...
7166,2021-07-15,434.809998,435.529999,432.720001,434.750000,55126400,0.0,0
7167,2021-07-16,436.010010,436.059998,430.920013,431.339996,75784700,0.0,0
7168,2021-07-19,426.190002,431.410004,421.970001,424.970001,147987000,0.0,0
7169,2021-07-20,425.679993,432.420013,424.829987,431.059998,99501600,0.0,0


In [8]:
import pandas as pd
df = df.rename(columns={'Date':'ds', 'Close':'y'})
# df = pd.DataFrame()

# df['ds'] = hist.index
# df['y'] = hist.Close.values
df.head()

,ds,Open,High,Low,y,Volume,Dividends,Stock Splits
0,2005-11-15,10.496973,10.508061,10.458164,10.508061,36000,0.0,0.0
1,2005-11-16,10.395329,10.439683,10.395329,10.439683,80000,0.0,0.0
2,2005-11-17,10.439683,10.439683,10.439683,10.439683,0,0.0,0.0
3,2005-11-18,10.578286,10.578286,10.578286,10.578286,800,0.0,0.0
4,2005-11-21,10.561653,10.620790,10.561653,10.620790,8000,0.0,0.0


In [9]:
from prophet import Prophet

m = Prophet(daily_seasonality=False)
# m = Prophet()

m.fit(df)

In [10]:
future = m.make_future_dataframe(365, freq='D')
forecast = m.predict(future)

In [11]:
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2005-11-15,10.378369,8.414246,11.866488,10.378369,10.378369,-0.175850,-0.175850,-0.175850,-0.011547,-0.011547,-0.011547,-0.164303,-0.164303,-0.164303,0.0,0.0,0.0,10.202520
1,2005-11-16,10.382757,8.465282,11.984205,10.382757,10.382757,-0.169151,-0.169151,-0.169151,-0.015081,-0.015081,-0.015081,-0.154070,-0.154070,-0.154070,0.0,0.0,0.0,10.213606
2,2005-11-17,10.387145,8.531078,12.026081,10.387145,10.387145,-0.153804,-0.153804,-0.153804,-0.010339,-0.010339,-0.010339,-0.143465,-0.143465,-0.143465,0.0,0.0,0.0,10.233342
3,2005-11-18,10.391533,8.488764,11.971491,10.391533,10.391533,-0.152504,-0.152504,-0.152504,-0.019963,-0.019963,-0.019963,-0.132541,-0.132541,-0.132541,0.0,0.0,0.0,10.239029
4,2005-11-21,10.404697,8.547044,11.864921,10.404697,10.404697,-0.133251,-0.133251,-0.133251,-0.034673,-0.034673,-0.034673,-0.098578,-0.098578,-0.098578,0.0,0.0,0.0,10.271446


In [12]:
forecast[['ds', 'yhat','yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
4273,2022-05-30,48.900927,46.387292,51.262916
4274,2022-05-31,48.929713,46.360340,51.210343
4275,2022-06-01,48.930579,46.442634,51.153912
4276,2022-06-02,48.938398,46.512100,51.389975
4277,2022-06-03,48.930483,46.455777,51.329177


In [13]:
# fig1 = m.plot(forecast);

In [14]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast);

In [15]:
plot_components_plotly(m, forecast);

## Make function

In [33]:
import yfinance as yf
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly




def show_ticker(ticker, period='max'):
    
    # pull market data from yahoo finance
    data = yf.Ticker(ticker)
    hist = data.history(period, auto_adjust=True)
    # convert it to dataframe
    df = pd.DataFrame(hist).reset_index()

    return df

def make_forecast(ticker, periods):
    # prepare for prophet to run
    df = show_ticker(ticker)
    df_re = df.rename(columns={'Date' : 'ds', 'Close' : 'y'})
    m = Prophet(daily_seasonality=False)
    m.fit(df_re)
    future = m.make_future_dataframe(365, freq='D')
    forecast = m.predict(future)
    
    return m, forecast


In [34]:
show_ticker('DXJ')

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2006-06-16,32.884606,32.950139,32.884606,32.923927,37600,0.0,0
1,2006-06-19,33.028771,33.028771,32.819065,32.819065,20200,0.0,0
2,2006-06-20,32.766639,32.878044,32.766639,32.825619,21000,0.0,0
3,2006-06-21,32.681441,33.127068,32.681441,33.127068,1600,0.0,0
4,2006-06-22,33.127068,33.127068,33.127068,33.127068,0,0.0,0
...,...,...,...,...,...,...,...,...
3762,2021-05-27,60.930000,61.240002,60.930000,61.189999,340200,0.0,0
3763,2021-05-28,61.709999,61.799999,61.439999,61.439999,298500,0.0,0
3764,2021-06-01,61.410000,61.410000,60.990002,61.110001,576600,0.0,0
3765,2021-06-02,61.750000,61.840000,61.650002,61.750000,166000,0.0,0


In [35]:
(m, forecast) = make_forecast('DXJ', 365)
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2006-06-16,34.342561,30.824742,37.759020,34.342561,34.342561,0.046942,0.046942,0.046942,0.323086,0.323086,0.323086,-0.276144,-0.276144,-0.276144,0.0,0.0,0.0,34.389503
1,2006-06-19,34.359777,31.212152,37.825442,34.359777,34.359777,-0.078086,-0.078086,-0.078086,0.316171,0.316171,0.316171,-0.394257,-0.394257,-0.394257,0.0,0.0,0.0,34.281691
2,2006-06-20,34.365516,30.881401,37.686917,34.365516,34.365516,-0.047609,-0.047609,-0.047609,0.375512,0.375512,0.375512,-0.423121,-0.423121,-0.423121,0.0,0.0,0.0,34.317907
3,2006-06-21,34.371255,31.136732,37.683897,34.371255,34.371255,-0.084763,-0.084763,-0.084763,0.361482,0.361482,0.361482,-0.446246,-0.446246,-0.446246,0.0,0.0,0.0,34.286492
4,2006-06-22,34.376994,30.931746,37.614721,34.376994,34.376994,-0.119330,-0.119330,-0.119330,0.344163,0.344163,0.344163,-0.463493,-0.463493,-0.463493,0.0,0.0,0.0,34.257664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4127,2022-05-30,52.566441,47.405155,58.933682,47.616405,57.278205,0.906723,0.906723,0.906723,0.316171,0.316171,0.316171,0.590552,0.590552,0.590552,0.0,0.0,0.0,53.473163
4128,2022-05-31,52.569196,47.411904,59.002965,47.572286,57.311382,0.935033,0.935033,0.935033,0.375512,0.375512,0.375512,0.559521,0.559521,0.559521,0.0,0.0,0.0,53.504229
4129,2022-06-01,52.571951,47.649056,58.951312,47.528388,57.344558,0.885650,0.885650,0.885650,0.361482,0.361482,0.361482,0.524167,0.524167,0.524167,0.0,0.0,0.0,53.457601
4130,2022-06-02,52.574706,47.543694,59.444033,47.518767,57.377735,0.828752,0.828752,0.828752,0.344163,0.344163,0.344163,0.484589,0.484589,0.484589,0.0,0.0,0.0,53.403458


In [36]:
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)

In [ ]:
forecast